Credit: 
https://www.kaggle.com/nicohrubec/lgb-baseline-with-groupkfold
https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline

In [ ]:
import numpy as np
import pandas as pd
import os, gc
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr as p
from lightgbm import LGBMRegressor as lgbr
from catboost import CatBoostRegressor as cbr

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline
params = {
        'n_estimators':1000,
        'learning_rate':0.1,
        "objective": "regression",
        "metric": "rmse",
        'boosting_type': "gbdt",
        'verbosity': -1,
        'n_jobs': -1, 
        'seed': 21,
        'lambda_l1': 1.1895057699067542, 
        'lambda_l2': 1.9079686837880768e-08, 
        'num_leaves': 112, 
        'subsample':0.8,
        'feature_fraction': 0.6259927292757151, 
        'bagging_fraction': 0.9782210574588895, 
        'bagging_freq': 1, 
        'n_estimators': 306, 
        'max_depth': 12, 
        'max_bin': 494, 
        'min_data_in_leaf': 366,
        'colsample_bytree': None,
        'subsample_freq': None,
        'min_child_samples': None,
        'reg_lambda': None,
        'reg_alpha': None,
    }

In [ ]:
def run(train):
    fold_scores = []
    for fold, (trn_idx, val_idx) in enumerate(kfold.split(train[feats], train['investment_id'])):
        X_train, y_train = train.iloc[trn_idx][feats], train.iloc[trn_idx][target]
        X_val, y_val = train.iloc[val_idx][feats], train.iloc[val_idx][target]

        model_lgbr = lgbr(**params)
        model_lgbr.fit(X_train, y_train, eval_metric='rmse', eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=100)
        models.append(model_lgbr)
        
        model_cbr = cbr()
        model_cbr.fit(X_train, y_train, eval_set=[(X_val, y_val)], use_best_model = True, early_stopping_rounds=100, verbose = False)
        models.append(model_cbr)
        
        y_pred = ( model_lgbr.predict(X_val) + model_cbr.predict(X_val) ) / 2.0

        score = p(y_pred, y_val)[0]
        print(f"Fold {fold + 1}: {score}")
        fold_scores.append(score)
        
        del model_lgbr, model_cbr, y_pred, score
        gc.collect()
    
    print(f"Overall score: {np.mean(fold_scores)}")

In [ ]:
train = pd.read_pickle('../input/ump195gb/train.pkl')

In [ ]:
models = []
n_splits=5
feats = [f for f in train.columns if f not in ['time_id', 'row_id', 'target']]
target = 'target'
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
run(train[-1500000:])

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[feats]
    pred = np.zeros(len(test_df))
    
    for i in range(len(models)):
        pred += models[i].predict(test_df)
    
    pred /= len(models)
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df)